In [73]:
import pandas as pd
import boto3
from io import StringIO, BytesIO
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

In [59]:
arg_date = '2022-01-04'
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)


In [60]:
s3 = boto3.resource('s3')

bucket = s3.Bucket('xetra-1234')

bucket_obj1 = bucket.objects.filter(Prefix='dataset/2021-03-15/')
bucket_obj2 = bucket.objects.filter(Prefix='dataset/2021-03-16/')

desired_dates = ['2022-01-03', '2022-01-04']


In [61]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_dt] #filter objects of the bucket
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR10.csv'),
 s3.Object

In [62]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

# Read multiple files

In [63]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in bucket.objects.all():
    for date in desired_dates:
        if date in obj.key:
            csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
            data = StringIO(csv_obj)
            df = pd.read_csv(data, delimiter=',')
            df_all = pd.concat([df_all, df], ignore_index=True, sort=False).dropna(axis=1, how='all')
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414,2
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649,3
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011,36
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308,22
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239075,DE0007164600,SAP,SAP SE O.N.,Common stock,EUR,2505077,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000,3
239076,MT0000580101,M8G,MEDIA AND GAMES INV. EO 1,Common stock,EUR,2509636,2022-01-04,16:44,4.118,4.118,4.118,4.118,24,2
239077,DE000A0Z23Q5,ADN1,ADESSO SE INH O.N.,Common stock,EUR,2504440,2022-01-04,16:45,196.800,196.800,196.800,196.800,7,2
239078,DE000A3E5ES0,BIO0,BIOTEST AG ST O.N.Z.VERK.,Common stock,EUR,6973997,2022-01-04,20:30,43.000,43.000,43.000,43.000,0,1


In [64]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414
1,DE000A0DJ6J9,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649
2,DE000A0D6554,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011
3,DE000A0D9PT0,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308
4,DE000A0HN5C6,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897
...,...,...,...,...,...,...,...,...
239075,DE0007164600,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000
239076,MT0000580101,2022-01-04,16:44,4.118,4.118,4.118,4.118,24
239077,DE000A0Z23Q5,2022-01-04,16:45,196.800,196.800,196.800,196.800,7
239078,DE000A3E5ES0,2022-01-04,20:30,43.000,43.000,43.000,43.000,0


In [65]:
df_all.dropna(inplace=True)
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414
1,DE000A0DJ6J9,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649
2,DE000A0D6554,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011
3,DE000A0D9PT0,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308
4,DE000A0HN5C6,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897
...,...,...,...,...,...,...,...,...
239075,DE0007164600,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000
239076,MT0000580101,2022-01-04,16:44,4.118,4.118,4.118,4.118,24
239077,DE000A0Z23Q5,2022-01-04,16:45,196.800,196.800,196.800,196.800,7
239078,DE000A3E5ES0,2022-01-04,20:30,43.000,43.000,43.000,43.000,0


# Transformations

## Get opening price per ISIN and day

In [66]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414,14.760
1,DE000A0DJ6J9,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649,37.640
2,DE000A0D6554,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011,13.990
3,DE000A0D9PT0,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308,180.000
4,DE000A0HN5C6,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897,37.280
...,...,...,...,...,...,...,...,...,...
239075,DE0007164600,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000,124.900
239076,MT0000580101,2022-01-04,16:44,4.118,4.118,4.118,4.118,24,4.294
239077,DE000A0Z23Q5,2022-01-04,16:45,196.800,196.800,196.800,196.800,7,211.000
239078,DE000A3E5ES0,2022-01-04,20:30,43.000,43.000,43.000,43.000,0,43.000


In [67]:
df_all[df_all['ISIN']=='DE000A0HN5C6']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
4,DE000A0HN5C6,2022-01-03,08:00,37.28,37.28,37.28,37.28,2897,37.28
161,DE000A0HN5C6,2022-01-03,08:01,37.28,37.33,37.28,37.33,1000,37.28
281,DE000A0HN5C6,2022-01-03,08:02,37.57,37.61,37.55,37.61,2259,37.28
579,DE000A0HN5C6,2022-01-03,08:03,37.69,37.73,37.65,37.65,2477,37.28
745,DE000A0HN5C6,2022-01-03,08:04,37.66,37.86,37.65,37.85,2900,37.28
...,...,...,...,...,...,...,...,...,...
234503,DE000A0HN5C6,2022-01-04,16:26,37.27,37.28,37.26,37.26,884,38.02
234792,DE000A0HN5C6,2022-01-04,16:27,37.28,37.29,37.28,37.28,349,38.02
235111,DE000A0HN5C6,2022-01-04,16:28,37.28,37.28,37.27,37.27,148,38.02
235460,DE000A0HN5C6,2022-01-04,16:29,37.27,37.33,37.27,37.33,1055,38.02


## Get closing price per ISIN and day

In [68]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['EndPrice'].transform('last')
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,AT0000A0E9W5,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414,14.760,15.620
1,DE000A0DJ6J9,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649,37.640,38.480
2,DE000A0D6554,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011,13.990,14.470
3,DE000A0D9PT0,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308,180.000,186.600
4,DE000A0HN5C6,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897,37.280,37.640
...,...,...,...,...,...,...,...,...,...,...
239075,DE0007164600,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000,124.900,124.380
239076,MT0000580101,2022-01-04,16:44,4.118,4.118,4.118,4.118,24,4.294,4.118
239077,DE000A0Z23Q5,2022-01-04,16:45,196.800,196.800,196.800,196.800,7,211.000,196.800
239078,DE000A3E5ES0,2022-01-04,20:30,43.000,43.000,43.000,43.000,0,43.000,43.000


## Aggregations

In [69]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index = False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-01-03,36.550,37.400,36.3500,37.400,660
1,AT000000STR1,2022-01-04,37.750,37.850,37.7500,37.850,27
2,AT00000FACC2,2022-01-03,7.390,7.650,7.3900,7.670,1045
3,AT00000FACC2,2022-01-04,7.860,7.790,7.7900,7.950,681
4,AT0000606306,2022-01-03,25.800,25.860,25.8000,26.000,455
...,...,...,...,...,...,...,...
6348,XS2314659447,2022-01-04,8.495,8.548,8.4555,8.548,0
6349,XS2314660700,2022-01-03,17.967,17.867,17.8670,18.097,19
6350,XS2314660700,2022-01-04,18.305,18.606,18.3050,18.606,0
6351,XS2376095068,2022-01-03,41.504,41.044,41.0000,41.832,2


## Percent Change Prev Closing

In [70]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price
0,AT000000STR1,2022-01-03,36.550,37.400,36.3500,37.400,660,NaN
1,AT000000STR1,2022-01-04,37.750,37.850,37.7500,37.850,27,37.4000
2,AT00000FACC2,2022-01-03,7.390,7.650,7.3900,7.670,1045,NaN
3,AT00000FACC2,2022-01-04,7.860,7.790,7.7900,7.950,681,7.6500
4,AT0000606306,2022-01-03,25.800,25.860,25.8000,26.000,455,NaN
...,...,...,...,...,...,...,...,...
6348,XS2314659447,2022-01-04,8.495,8.548,8.4555,8.548,0,8.6615
6349,XS2314660700,2022-01-03,17.967,17.867,17.8670,18.097,19,NaN
6350,XS2314660700,2022-01-04,18.305,18.606,18.3050,18.606,0,17.8670
6351,XS2376095068,2022-01-03,41.504,41.044,41.0000,41.832,2,NaN


In [71]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price']*100
df_all.drop(columns=['prev_closing_price'], inplace=True)
df_all = df_all.round(decimals = 2)
df_all = df_all[df_all.Date >= arg_date] # we filter from the arg_date forward
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
3,AT00000FACC2,2022-01-04,7.86,7.79,7.79,7.95,681,1.83
5,AT0000606306,2022-01-04,26.00,26.80,26.00,26.80,339,3.63
7,AT0000609607,2022-01-04,13.78,13.72,13.72,13.78,400,0.59
9,AT0000644505,2022-01-04,123.20,123.20,123.20,123.20,13,3.53
...,...,...,...,...,...,...,...,...
6344,XS2265370234,2022-01-04,16.22,16.52,15.97,16.62,426,2.70
6346,XS2284324667,2022-01-04,32.66,32.87,32.66,32.87,0,1.78
6348,XS2314659447,2022-01-04,8.49,8.55,8.46,8.55,0,-1.31
6350,XS2314660700,2022-01-04,18.30,18.61,18.30,18.61,0,4.14


# Write to S3

In [74]:
key = 'report_'+ datetime.today().strftime("%Y%m%d_%H%m%S")+'.parquet'
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index = False)
bucket_target = s3.Bucket('etl-pipelines-kspyrikos')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='etl-pipelines-kspyrikos', key='report_20231118_191119.parquet')

## Read the uploaded file

In [75]:
for obj in bucket_target.objects.all():
    print(obj.key)

report_20231118_191119.parquet


In [79]:
prq_obj = bucket_target.Object(key='report_20231118_191119.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data) # I can use 'read_parquet', because I used 'BytesIO'
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
1,AT00000FACC2,2022-01-04,7.86,7.79,7.79,7.95,681,1.83
2,AT0000606306,2022-01-04,26.00,26.80,26.00,26.80,339,3.63
3,AT0000609607,2022-01-04,13.78,13.72,13.72,13.78,400,0.59
4,AT0000644505,2022-01-04,123.20,123.20,123.20,123.20,13,3.53
...,...,...,...,...,...,...,...,...
3155,XS2265370234,2022-01-04,16.22,16.52,15.97,16.62,426,2.70
3156,XS2284324667,2022-01-04,32.66,32.87,32.66,32.87,0,1.78
3157,XS2314659447,2022-01-04,8.49,8.55,8.46,8.55,0,-1.31
3158,XS2314660700,2022-01-04,18.30,18.61,18.30,18.61,0,4.14
